In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [8]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    # RandomZoom(0.2, 0.2)
])

In [9]:
model = Sequential()

model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(2e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(2e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(3e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(3e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
3095/3095 - 157s - loss: 0.5358 - accuracy: 0.8228 - val_loss: 0.4335 - val_accuracy: 0.8483
Epoch 2/10
3095/3095 - 153s - loss: 0.3804 - accuracy: 0.8717 - val_loss: 0.6847 - val_accuracy: 0.7817
Epoch 3/10
3095/3095 - 153s - loss: 0.3040 - accuracy: 0.8993 - val_loss: 1.4041 - val_accuracy: 0.7703
Epoch 4/10
3095/3095 - 153s - loss: 0.2727 - accuracy: 0.9113 - val_loss: 0.4182 - val_accuracy: 0.8535
Epoch 5/10
3095/3095 - 153s - loss: 0.2610 - accuracy: 0.9153 - val_loss: 1.2211 - val_accuracy: 0.7467
Epoch 6/10
3095/3095 - 153s - loss: 0.2543 - accuracy: 0.9179 - val_loss: 1.4389 - val_accuracy: 0.7442
Epoch 7/10
3095/3095 - 153s - loss: 0.2486 - accuracy: 0.9212 - val_loss: 0.8892 - val_accuracy: 0.8874
Epoch 8/10
3095/3095 - 153s - loss: 0.2446 - accuracy: 0.9226 - val_loss: 0.2530 - val_accuracy: 0.9265
Epoch 9/10
3095/3095 - 153s - loss: 0.2419 - accuracy: 0.9238 - val_loss: 0.3106 - val_accuracy: 0.8926
Epoch 10/10
3095/3095 - 153s - loss: 0.2419 - accuracy: 0.9241 -

In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
1548/1548 - 154s - loss: 0.1953 - accuracy: 0.9422 - val_loss: 0.2062 - val_accuracy: 0.9351
Epoch 2/10
1548/1548 - 151s - loss: 0.1855 - accuracy: 0.9452 - val_loss: 0.1875 - val_accuracy: 0.9419
Epoch 3/10
1548/1548 - 151s - loss: 0.1798 - accuracy: 0.9460 - val_loss: 0.1928 - val_accuracy: 0.9405
Epoch 4/10
1548/1548 - 151s - loss: 0.1749 - accuracy: 0.9472 - val_loss: 0.1792 - val_accuracy: 0.9438
Epoch 5/10
1548/1548 - 151s - loss: 0.1722 - accuracy: 0.9475 - val_loss: 0.1783 - val_accuracy: 0.9424
Epoch 6/10
1548/1548 - 151s - loss: 0.1696 - accuracy: 0.9481 - val_loss: 0.1849 - val_accuracy: 0.9379
Epoch 7/10
1548/1548 - 151s - loss: 0.1668 - accuracy: 0.9495 - val_loss: 0.1710 - val_accuracy: 0.9454
Epoch 8/10
1548/1548 - 151s - loss: 0.1636 - accuracy: 0.9503 - val_loss: 0.1761 - val_accuracy: 0.9418
Epoch 9/10
1548/1548 - 151s - loss: 0.1628 - accuracy: 0.9501 - val_loss: 0.2182 - val_accuracy: 0.9270
Epoch 10/10
1548/1548 - 151s - loss: 0.1610 - accuracy: 0.9506 -

In [12]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 156s - loss: 0.1470 - accuracy: 0.9556 - val_loss: 0.1499 - val_accuracy: 0.9527
Epoch 2/10
774/774 - 151s - loss: 0.1451 - accuracy: 0.9560 - val_loss: 0.1575 - val_accuracy: 0.9501
Epoch 3/10
774/774 - 151s - loss: 0.1442 - accuracy: 0.9565 - val_loss: 0.1711 - val_accuracy: 0.9433
Epoch 4/10
774/774 - 152s - loss: 0.1422 - accuracy: 0.9574 - val_loss: 0.1575 - val_accuracy: 0.9499
Epoch 5/10
774/774 - 151s - loss: 0.1426 - accuracy: 0.9572 - val_loss: 0.1898 - val_accuracy: 0.9365
Epoch 6/10
774/774 - 151s - loss: 0.1419 - accuracy: 0.9573 - val_loss: 0.1535 - val_accuracy: 0.9529
Epoch 7/10
774/774 - 151s - loss: 0.1406 - accuracy: 0.9581 - val_loss: 0.1459 - val_accuracy: 0.9554
Epoch 8/10
774/774 - 151s - loss: 0.1402 - accuracy: 0.9582 - val_loss: 0.1503 - val_accuracy: 0.9532
Epoch 9/10
774/774 - 151s - loss: 0.1399 - accuracy: 0.9578 - val_loss: 0.1566 - val_accuracy: 0.9502
Epoch 10/10
774/774 - 152s - loss: 0.1388 - accuracy: 0.9587 - val_loss: 0.1572 - 

In [13]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 154s - loss: 0.1583 - accuracy: 0.9505 - val_loss: 0.1730 - val_accuracy: 0.9409
Epoch 2/2
3095/3095 - 153s - loss: 0.1577 - accuracy: 0.9509 - val_loss: 0.1748 - val_accuracy: 0.9418


In [14]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 153s - loss: 0.1390 - accuracy: 0.9579 - val_loss: 0.1555 - val_accuracy: 0.9500
Epoch 2/10
774/774 - 152s - loss: 0.1372 - accuracy: 0.9592 - val_loss: 0.1840 - val_accuracy: 0.9386
Epoch 3/10
774/774 - 152s - loss: 0.1354 - accuracy: 0.9597 - val_loss: 0.1432 - val_accuracy: 0.9562
Epoch 4/10
774/774 - 152s - loss: 0.1348 - accuracy: 0.9597 - val_loss: 0.1579 - val_accuracy: 0.9497
Epoch 5/10
774/774 - 152s - loss: 0.1337 - accuracy: 0.9598 - val_loss: 0.1524 - val_accuracy: 0.9517
Epoch 6/10
774/774 - 152s - loss: 0.1343 - accuracy: 0.9599 - val_loss: 0.1456 - val_accuracy: 0.9550
Epoch 7/10
774/774 - 152s - loss: 0.1332 - accuracy: 0.9598 - val_loss: 0.1625 - val_accuracy: 0.9473
Epoch 8/10
774/774 - 152s - loss: 0.1326 - accuracy: 0.9604 - val_loss: 0.1373 - val_accuracy: 0.9585
Epoch 9/10
774/774 - 152s - loss: 0.1332 - accuracy: 0.9596 - val_loss: 0.1687 - val_accuracy: 0.9450
Epoch 10/10
774/774 - 152s - loss: 0.1312 - accuracy: 0.9608 - val_loss: 0.1597 - 

In [15]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 163s - loss: 0.1271 - accuracy: 0.9618 - val_loss: 0.1459 - val_accuracy: 0.9535
Epoch 2/10
387/387 - 153s - loss: 0.1271 - accuracy: 0.9624 - val_loss: 0.1405 - val_accuracy: 0.9557
Epoch 3/10
387/387 - 154s - loss: 0.1258 - accuracy: 0.9627 - val_loss: 0.1487 - val_accuracy: 0.9531
Epoch 4/10
387/387 - 154s - loss: 0.1261 - accuracy: 0.9629 - val_loss: 0.1418 - val_accuracy: 0.9558
Epoch 5/10
387/387 - 154s - loss: 0.1256 - accuracy: 0.9624 - val_loss: 0.1439 - val_accuracy: 0.9541
Epoch 6/10
387/387 - 154s - loss: 0.1254 - accuracy: 0.9626 - val_loss: 0.1441 - val_accuracy: 0.9538
Epoch 7/10
387/387 - 153s - loss: 0.1254 - accuracy: 0.9632 - val_loss: 0.1486 - val_accuracy: 0.9519
Epoch 8/10
387/387 - 154s - loss: 0.1248 - accuracy: 0.9631 - val_loss: 0.1507 - val_accuracy: 0.9516
Epoch 9/10
387/387 - 154s - loss: 0.1260 - accuracy: 0.9623 - val_loss: 0.1444 - val_accuracy: 0.9540
Epoch 10/10
387/387 - 154s - loss: 0.1258 - accuracy: 0.9626 - val_loss: 0.1435 - 

In [16]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 154s - loss: 0.1266 - accuracy: 0.9622 - val_loss: 0.1546 - val_accuracy: 0.9504
Epoch 2/10
387/387 - 154s - loss: 0.1269 - accuracy: 0.9620 - val_loss: 0.1471 - val_accuracy: 0.9541
Epoch 3/10
387/387 - 154s - loss: 0.1259 - accuracy: 0.9621 - val_loss: 0.1687 - val_accuracy: 0.9451
Epoch 4/10
387/387 - 153s - loss: 0.1260 - accuracy: 0.9622 - val_loss: 0.1661 - val_accuracy: 0.9457
Epoch 5/10
387/387 - 154s - loss: 0.1253 - accuracy: 0.9628 - val_loss: 0.1432 - val_accuracy: 0.9551
Epoch 6/10
387/387 - 154s - loss: 0.1251 - accuracy: 0.9626 - val_loss: 0.1376 - val_accuracy: 0.9576
Epoch 7/10
387/387 - 154s - loss: 0.1249 - accuracy: 0.9626 - val_loss: 0.1554 - val_accuracy: 0.9501
Epoch 8/10
387/387 - 154s - loss: 0.1240 - accuracy: 0.9630 - val_loss: 0.1635 - val_accuracy: 0.9482
Epoch 9/10
387/387 - 154s - loss: 0.1247 - accuracy: 0.9627 - val_loss: 0.1502 - val_accuracy: 0.9519
Epoch 10/10
387/387 - 154s - loss: 0.1242 - accuracy: 0.9629 - val_loss: 0.1380 - 